In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

import re
from nltk.tokenize  import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.utils import shuffle


from sklearn.model_selection import train_test_split

import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers.core import Flatten
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy , CategoricalCrossentropy

from tensorflow.keras.utils import to_categorical

from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("../input/twitter-sentiment-dataset/Twitter_Data.csv")
df.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [3]:
df['category'].mask(df['category'] == -1,'negative',  inplace=True)
df['category'].mask(df['category'] == 0,'normal',  inplace=True)
df['category'].mask(df['category'] == 1,'positive',  inplace=True)

In [4]:
df.head()

,clean_text,category
0,when modi promised “minimum government maximum...,negative
1,talk all the nonsense and continue all the dra...,normal
2,what did just say vote for modi welcome bjp t...,positive
3,asking his supporters prefix chowkidar their n...,positive
4,answer who among these the most powerful world...,positive


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162980 entries, 0 to 162979
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   clean_text  162976 non-null  object
 1   category    162973 non-null  object
dtypes: object(2)
memory usage: 2.5+ MB


In [6]:
df.category.unique()

array(['negative', 'normal', 'positive', nan], dtype=object)

In [7]:
df.isna().sum()

clean_text    4
category      7
dtype: int64

In [8]:
df= df.dropna()

In [9]:
df.duplicated().sum()

0

In [10]:
df['category'].value_counts()

positive    72249
normal      55211
negative    35509
Name: category, dtype: int64

In [11]:
dist = df['category'].value_counts()
import plotly.graph_objects as go
def ditribution_plot(x,y,name):
    fig = go.Figure([
        go.Bar(x=x, y=y)
    ])

    fig.update_layout(title_text=name)
    fig.show()
ditribution_plot(x= dist.index, y= dist.values, name= 'Class Distribution train')

In [12]:
'''def clean_text(df, field):
    df[field] = df[field].str.replace(r"@"," at ")
    df[field] = df[field].str.replace("#[^a-zA-Z0-9_]+"," ")
    df[field] = df[field].str.replace(r"[^a-zA-Z(),\"'\n_]"," ")
    df[field] = df[field].str.replace(r"http\S+","")
    df[field] = df[field].str.lower()
    return df

clean_text(df,'tweet')'''

'def clean_text(df, field):\n    df[field] = df[field].str.replace(r"@"," at ")\n    df[field] = df[field].str.replace("#[^a-zA-Z0-9_]+"," ")\n    df[field] = df[field].str.replace(r"[^a-zA-Z(),"\'\n_]"," ")\n    df[field] = df[field].str.replace(r"http\\S+","")\n    df[field] = df[field].str.lower()\n    return df\n\nclean_text(df,\'tweet\')'

In [13]:
'''stem = PorterStemmer()
df['clean_text'] = [stem.stem(word) for word in df['clean_text'] if not word in stopwords.words('english')]'''

"stem = PorterStemmer()\ndf['clean_text'] = [stem.stem(word) for word in df['clean_text'] if not word in stopwords.words('english')]"

In [14]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['clean_text'])
df['clean_text'] = tokenizer.texts_to_sequences(df['clean_text'])
df['clean_text']

0         [42, 1, 307, 42492, 66, 1726, 42493, 1119, 40,...
1         [286, 16, 2, 1104, 3, 718, 16, 2, 809, 7, 38, ...
2         [24, 53, 41, 99, 38, 4, 1, 1003, 21, 508, 5, 5...
3         [383, 19, 496, 4730, 113, 43, 1221, 1, 53, 126...
4         [390, 22, 813, 79, 2, 173, 926, 127, 136, 124,...
                                ...                        
162975    [32, 79, 5172, 474, 570, 2429, 1, 6, 2715, 17,...
162976    [305, 363, 529, 13161, 10149, 24, 36, 1, 1195,...
162977        [53, 5, 1718, 234, 2583, 7526, 118, 191, 397]
162978    [63, 183, 558, 360, 192, 9, 1, 924, 558, 15, 6...
162979    [14, 5, 260, 686, 36, 26, 35329, 118, 4882, 10...
Name: clean_text, Length: 162969, dtype: object

In [15]:
df['category'].mask(df['category'] == 'negative',-1,  inplace=True)
df['category'].mask(df['category'] == 'normal',0,  inplace=True)
df['category'].mask(df['category'] == 'positive',1,  inplace=True)
df['category']

0         -1
1          0
2          1
3          1
4          1
          ..
162975    -1
162976    -1
162977     0
162978     0
162979     1
Name: category, Length: 162969, dtype: object

In [16]:
X_train, X_test, y_train, y_test = train_test_split(df['clean_text'],df['category'], test_size=0.2, random_state=40)

In [17]:
print('X_train:',len(X_train))
print('y_train:',len(y_train))
print('X_test:',len(X_test))
print('y_test:',len(y_test))

X_train: 130375
y_train: 130375
X_test: 32594
y_test: 32594


In [18]:
X_train = pad_sequences( X_train, maxlen=100 ,dtype='float32')
X_test = pad_sequences( X_test, maxlen=100 ,dtype='float32')

In [19]:
len(tokenizer.index_word)

113678

In [20]:
model_1 = Sequential()
model_1.add(Embedding(len(tokenizer.index_word)+1, input_length= 64 ,output_dim =50))
model_1.add(Bidirectional(LSTM(100)))
model_1.add(Flatten())
model_1.add(Dense(250, activation='relu'))
model_1.add(Dropout(0.2))
model_1.add(Dense(1, activation='sigmoid'))

model_1.compile("adam", loss= 'binary_crossentropy' ,metrics=["accuracy"])
model_1.summary()

2022-07-03 20:21:49.368163: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-03 20:21:49.481379: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-03 20:21:49.482200: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-03 20:21:49.483495: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 64, 50)            5683950   
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               120800    
_________________________________________________________________
flatten (Flatten)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 250)               50250     
_________________________________________________________________
dropout (Dropout)            (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 251       
Total params: 5,855,251
Trainable params: 5,855,251
Non-trainable params: 0
______________________________________________

In [21]:
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor="val_loss",patience=5,verbose=True)

In [22]:
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)

y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

In [23]:
history_1 = model_1.fit(X_train , y_train ,batch_size=64, epochs=7,
                    validation_data=(X_test , y_test),callbacks=[early_stop])

2022-07-03 20:21:52.690787: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/7


2022-07-03 20:21:56.454932: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


2038/2038 [==============================] - 38s 16ms/step - loss: -2459.0239 - accuracy: 0.5261 - val_loss: -11827.6797 - val_accuracy: 0.6721
Epoch 2/7
2038/2038 [==============================] - 32s 16ms/step - loss: -22885.1289 - accuracy: 0.5979 - val_loss: -20169.3848 - val_accuracy: 0.5876
Epoch 3/7
2038/2038 [==============================] - 35s 17ms/step - loss: -73696.3203 - accuracy: 0.6447 - val_loss: -102316.1875 - val_accuracy: 0.6949
Epoch 4/7
2038/2038 [==============================] - 32s 16ms/step - loss: -158957.2812 - accuracy: 0.6409 - val_loss: -165186.8125 - val_accuracy: 0.6667
Epoch 5/7
2038/2038 [==============================] - 32s 16ms/step - loss: -215412.7500 - accuracy: 0.6117 - val_loss: -291410.0938 - val_accuracy: 0.6567
Epoch 6/7
2038/2038 [==============================] - 33s 16ms/step - loss: -400651.3750 - accuracy: 0.6570 - val_loss: -427005.3438 - val_accuracy: 0.6431
Epoch 7/7
2038/2038 [==============================] - 32s 16ms/step - los

In [24]:
y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)

In [25]:
model = Sequential()
model.add(Embedding(len(tokenizer.index_word)+1, input_length= 64 ,output_dim =100))
model.add(LSTM(100))
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='sigmoid'))

model.compile("adam", loss= 'categorical_crossentropy' ,metrics=["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 64, 100)           11367900  
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               12928     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,461,615
Trainable params: 11,461,615
Non-trainable params: 0
_________________________________________________________________


In [26]:
history = model.fit(X_train , y_train ,batch_size=256, epochs=4,
                    validation_data=(X_test , y_test),callbacks=[early_stop])

Epoch 1/4
510/510 [==============================] - 10s 16ms/step - loss: 0.3618 - accuracy: 0.8587 - val_loss: 0.1681 - val_accuracy: 0.9510
Epoch 2/4
510/510 [==============================] - 8s 15ms/step - loss: 0.1082 - accuracy: 0.9673 - val_loss: 0.1403 - val_accuracy: 0.9588
Epoch 3/4
510/510 [==============================] - 8s 16ms/step - loss: 0.0644 - accuracy: 0.9797 - val_loss: 0.1366 - val_accuracy: 0.9603
Epoch 4/4
510/510 [==============================] - 8s 15ms/step - loss: 0.0440 - accuracy: 0.9857 - val_loss: 0.1290 - val_accuracy: 0.9615


In [27]:
results = model.evaluate(X_test, y_test, batch_size=64)

510/510 [==============================] - 2s 4ms/step - loss: 0.1290 - accuracy: 0.9615
